In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 29 02:23:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import tensorflow as tf
import json
import pandas as pd
import numpy as np
import os
import torch as t
import datetime

In [4]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 14.3 MB/s 
     |████████████████████████████████| 895 kB 65.5 MB/s 
     |████████████████████████████████| 3.3 MB 73.8 MB/s 
     |████████████████████████████████| 596 kB 83.2 MB/s 
     |████████████████████████████████| 59 kB 8.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import transformers

In [7]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", device=0) ## device = 0 forces it to use GPU

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
# input_text = ["this resturant has very good tacos. The music they play is my jam. I love that they have many drink options, especially during happy hour. The price is just unheard of! This is a must visit place if you stopped by Berkeley!"]
# candidate_labels = ["expensive", 
#                     "small portion size", 
#                     "bad customer service"]

In [ ]:
# result = classifier(input_text, candidate_labels, multi_label=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [30]:
#Inputs
#df: dataframe with reviews (column name with reviews should be "text")
#tags: the tags you want to find in the reviews (Eg : ['expensive','costly','not worth the price'])
def zsl_processor(df,tags):
  summary_label=pd.DataFrame()
  for i in range(len(df)):
    rev_id=df['review_id'][i]
    msg=df['text'][i]
    result = classifier(msg, tags, multi_label=True)
    df_results=pd.DataFrame(result)
    for j in range(len(tags)):
      temp_rev_label=pd.DataFrame({'review_id':[rev_id],'text':[msg],'label':[df_results.labels[j]],'score':[df_results.scores[j]]})
      summary_label=summary_label.append(temp_rev_label)
  return summary_label

In [31]:
test_df = pd.read_excel('all_labelled_w_stars.xlsx')
test_df = test_df.sample(200, random_state=200).reset_index(drop=True)

In [20]:
test_df.columns

Index(['review_id', 'user_id', 'business_id', 'text', 'expensive',
       'bad_customer_service', 'small_portion_size', 'poor_ambience',
       'bad_food_taste', 'stars'],
      dtype='object')

In [32]:
start = datetime.datetime.now()
test = zsl_processor(test_df, ['expensive','not worth the price'])
time_elapsed=datetime.datetime.now()
print('Length:', len(test_df), ",    time elapsed:", time_elapsed)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:910: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


Length: 200 ,    time elapsed: 2021-11-29 02:40:49.576853


In [33]:
test.head()

,review_id,text,label,score
0,0kDXtCwOF13MVyTYL9RW7Q,Finally got around to trying this place after ...,expensive,0.635648
0,0kDXtCwOF13MVyTYL9RW7Q,Finally got around to trying this place after ...,not worth the price,0.002072
0,PTq0MYH-KrcArO74KIgwjQ,Our 4th time to Sago and couldn't believe we g...,expensive,0.501741
0,PTq0MYH-KrcArO74KIgwjQ,Our 4th time to Sago and couldn't believe we g...,not worth the price,0.023862
0,RqCqZVW1OZQGWm30AeaGfw,"Very cozy, great drink menu, and friendly peop...",expensive,0.015372
